<a href="https://colab.research.google.com/github/RonSK03/CNN_Food101/blob/main/CNN_PROJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
kmader_food41_path = kagglehub.dataset_download('kmader/food41')

print('Data source import complete.')


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kmader/food41")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from keras.layers import Flatten

In [ ]:
import torch.nn as nn

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.convblock1=nn.Conv2d(3, 32, 3, 1, 1 )
        self.batchnorm1=nn.BatchNorm2d(32)
        self.relu1=nn.ReLU()
        self.maxp1=nn.MaxPool2d(3,2,1)

        self.convblock2=nn.Conv2d(32, 64, 3, 1, 1 )
        self.batchnorm2=nn.BatchNorm2d(64)
        self.relu2=nn.ReLU()
        self.maxp2=nn.MaxPool2d(3,2,1)

        self.convblock3=nn.Conv2d(64, 128, 3, 1, 1 )
        self.batchnorm3=nn.BatchNorm2d(128)
        self.relu3=nn.ReLU()
        self.maxp3=nn.MaxPool2d(3,2,1)

        self.convblock4=nn.Conv2d(128, 256, 3, 1, 1 )
        self.batchnorm4=nn.BatchNorm2d(256)
        self.relu4_=nn.ReLU()
        self.maxp4=nn.MaxPool2d(3,2,1)

        self.convblock5=nn.Conv2d(256, 512, 3, 1, 1 )
        self.batchnorm5=nn.BatchNorm2d(512)
        self.relu5_=nn.ReLU()
        self.maxp5=nn.MaxPool2d(3,2,1)



        self.flatten=nn.Flatten()
        #size getting code
        dummy_input = torch.zeros(1, 3, 128, 128)
        x = self.maxp1(self.relu1(self.batchnorm1(self.convblock1(dummy_input))))
        x = self.maxp2(self.relu2(self.batchnorm2(self.convblock2(x))))
        x = self.maxp3(self.relu3(self.batchnorm3(self.convblock3(x))))
        x=self.maxp4(self.relu4_(self.batchnorm4(self.convblock4(x))))
        x=self.maxp5(self.relu5_(self.batchnorm5(self.convblock5(x))))


        x = self.flatten(x)
        flattened_size = x.shape[1]


        self.layer1=nn.Linear(flattened_size, 1024)
        self.drop1=nn.Dropout(0.4)
        self.relu4=nn.ReLU()
        self.layer2=nn.Linear(1024, 512)
        self.drop2=nn.Dropout(0.4)
        self.relu5=nn.ReLU()
        self.layer3=nn.Linear(512, 256)
        self.drop3=nn.Dropout(0.4)
        self.relu6=nn.ReLU()
        self.layer4=nn.Linear(256, 101)
        self.drop4=nn.Dropout(0.4)
        self.relu7=nn.ReLU()



    def forward(self, x):
        x=self.maxp1(self.relu1(self.batchnorm1(self.convblock1(x))))
        x=self.maxp2(self.relu2(self.batchnorm2(self.convblock2(x))))
        x=self.maxp3(self.relu3(self.batchnorm3(self.convblock3(x))))
        x=self.maxp4(self.relu4_(self.batchnorm4(self.convblock4(x))))
        x=self.maxp5(self.relu5_(self.batchnorm5(self.convblock5(x))))
        x=self.flatten(x)
        x=self.layer4(self.relu6(self.drop3(self.layer3(self.relu5(self.drop2(self.layer2(self.relu4(self.drop1(self.layer1(x))))))))))
        return x


In [ ]:
import torchvision
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader


In [ ]:
transform=transforms.Compose([transforms.RandomHorizontalFlip(), #lets model know not just one orientation
                              transforms.RandomResizedCrop(128), #standard size for resnet is 224x224
                              transforms.ToTensor(),#converts the images to a tensor
                                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])

                              ])
#transforms required to convert image into form needed by ResNet 18 (normalize values used are the standard ones for ResNet18)

In [ ]:
print(path)

In [ ]:
net_data=ImageFolder('/kaggle/input/food41/images',transform=transform)

In [ ]:
len(net_data)

In [ ]:
train_size = int(0.8 * len(net_data))
test_size = len(net_data) - train_size

train_data, test_data = random_split(net_data, [train_size, test_size])
#splitting dataset as 80% training and 20% testing

In [ ]:
train_data_loaded=DataLoader(train_data, batch_size=16, shuffle=True)
#using dataloader to split the data into batches
test_data_loaded=DataLoader(test_data, batch_size=16, shuffle=True)

In [ ]:
train_data.dataset.classes

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device) #sending model to GPU for faster processing
if(torch.cuda.is_available()):
    print("Using GPU ")

In [ ]:
inputs, labels = next(iter(train_data_loaded))
inputs = inputs.to(device)
outputs = model(inputs)
print("Model output shape:", outputs.shape)


In [ ]:
train_data_loaded

In [ ]:
criterion=nn.CrossEntropyLoss() #loss/cost function checking how bad it is doing
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-4) #gradient descent alpha=lr


In [ ]:

train_loss=[]
test_loss=[]
epochacc=[]

for epochs in range(30):
    model.train()
    net_test_loss=0
    net_loss=0
    with torch.set_grad_enabled(True):
        for inputs, labels in train_data_loaded:
          inputs=inputs.to(device)
          labels=labels.to(device) #sending the inputs and labels to GPU
          optimizer.zero_grad() #makes the gradients of all weights 0 before next iteration
          outputs=model(inputs)
          _, preds=torch.max(outputs,1) #outputs returns a tensor of scores of all possibilities so we take the one with highest score
          loss=criterion(outputs, labels)
          loss.backward()#calulates loss and backpropagates
          optimizer.step()
          net_loss=net_loss+loss.item()
    avg_loss=net_loss/len(train_data_loaded)
    train_loss.append(avg_loss)
    model.eval() #sets model mode to evaluation/testin
    with torch.set_grad_enabled(False):
     running_corrects=0
     for inputs, labels in test_data_loaded:
        inputs=inputs.to(device)
        labels=labels.to(device)
        outputs=model(inputs)
        _, preds=torch.max(outputs, 1)
        loss=criterion(outputs, labels)
        running_corrects += torch.sum(preds == labels.data)
        net_test_loss=net_test_loss+loss.item()
     avg_test=net_test_loss/len(test_data_loaded)
    test_loss.append(avg_test)
    epoch_acc = running_corrects.double() /len(test_data_loaded.dataset)
    epochacc.append(epoch_acc.item())
    print(f"Accuracy={epoch_acc}")
    print(f"Epoch {epochs+1}/{30} -> Test Loss: {avg_test:.4f}, Accuracy: {epoch_acc:.4f}")



print("Test complete")
 #model training and testing for 30 iterations. saving model as above name


In [ ]:
inputs, labels = next(iter(train_data_loaded))
print("Input shape:", inputs.shape)  # Should be [B, 3, 128, 128]


In [ ]:
torch.cuda.empty_cache()


In [ ]:
print(labels[:5])
print(outputs.shape)  # should be (batch_size, 101)
